![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FMLOps%2FServing&file=notes.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/MLOps/Serving/notes.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FMLOps%2FServing%2Fnotes.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/MLOps/Serving/notes.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/MLOps/Serving/notes.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

---
# NOTES - IGNORE

---
# HOLD
---

In [70]:
#!pip install transformers tensorflow torch tf-keras tensorflow_hub tensorflow_text keras-hub keras-nlp -U -q --user

In [71]:
prompts = [
    'How do you play baseball?',
    'How do you play monopoly?',
    'How do you play chess?',
    'How do you play football?',
    'sports',
    'board games'
]

---

In [104]:
from transformers import AutoTokenizer, BertModel
import torch

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
pt_bert_model = BertModel.from_pretrained("google-bert/bert-base-uncased")

inputs = tokenizer(prompts[0], return_tensors="pt") # pt, tf, or np
outputs = pt_bert_model(**inputs)

last_hidden_states = outputs.last_hidden_state

In [105]:
inputs

{'input_ids': tensor([[ 101, 2129, 2079, 2017, 2377, 3598, 1029,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

CLS Token or classification token
BERT uses this to create a summary representation, capturing the overall meaning of the input sequence
Averaging, shown next, will treat all token equal and can dilute important key words or phrases

In [106]:
average_embedding = torch.mean(last_hidden_states, dim=1)

In [107]:
average_embedding[0, 0:10]

tensor([ 0.5702,  0.0869, -0.3377, -0.2114,  0.1338, -0.3392, -0.1137,  0.3860,
        -0.2058,  0.1436], grad_fn=<SliceBackward0>)

In [108]:
cls_embedding = last_hidden_states[:, 0, :]

In [109]:
cls_embedding[0, 0:10]

tensor([ 0.1670,  0.1293, -0.2888,  0.0183, -0.1663, -0.4358,  0.2780,  0.4109,
        -0.3145,  0.1231], grad_fn=<SliceBackward0>)

---

In [110]:
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tf_bert_model = TFBertModel.from_pretrained("bert-base-uncased")

inputs = tokenizer(prompts[0], return_tensors='tf')
outputs = tf_bert_model(**inputs)

last_hidden_states = outputs.last_hidden_state

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [111]:
inputs

{'input_ids': <tf.Tensor: shape=(1, 8), dtype=int32, numpy=array([[ 101, 2129, 2079, 2017, 2377, 3598, 1029,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 8), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 8), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [112]:
average_embedding = tf.reduce_mean(last_hidden_states, axis=1)

In [113]:
average_embedding[0, 0:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([ 0.57020056,  0.08687384, -0.33770347, -0.21141008,  0.13380972,
       -0.33915833, -0.11369041,  0.38600746, -0.20583639,  0.14364798],
      dtype=float32)>

In [114]:
cls_embedding = last_hidden_states[:, 0, :]

In [115]:
cls_embedding[0, 0:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([ 0.16698106,  0.12934545, -0.2888183 ,  0.01831304, -0.16634001,
       -0.43576777,  0.2779695 ,  0.4108801 , -0.31453484,  0.12307177],
      dtype=float32)>

---

In [133]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

tf_inputs = tokenizer(prompts[0], return_tensors='tf')
tf_outputs = tf_bert_model(**tf_inputs)

pt_inputs = tokenizer(prompts[0], return_tensors='pt')
pt_outputs = pt_bert_model(**pt_inputs)

In [134]:
tf_inputs

{'input_ids': <tf.Tensor: shape=(1, 8), dtype=int32, numpy=array([[ 101, 2129, 2079, 2017, 2377, 3598, 1029,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 8), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 8), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [135]:
pt_inputs

{'input_ids': tensor([[ 101, 2129, 2079, 2017, 2377, 3598, 1029,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [136]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tf_inputs = tokenizer(prompts[0], return_tensors='tf')
tf_outputs = tf_bert_model(**tf_inputs)

pt_inputs = tokenizer(prompts[0], return_tensors='pt')
pt_outputs = pt_bert_model(**pt_inputs)

In [137]:
tf_inputs

{'input_ids': <tf.Tensor: shape=(1, 8), dtype=int32, numpy=array([[ 101, 2129, 2079, 2017, 2377, 3598, 1029,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 8), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 8), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [138]:
pt_inputs

{'input_ids': tensor([[ 101, 2129, 2079, 2017, 2377, 3598, 1029,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

---

In [139]:
tf.saved_model.save(tf_bert_model, "./tf_bert_model") 

INFO:tensorflow:Assets written to: ./tf_bert_model/assets


INFO:tensorflow:Assets written to: ./tf_bert_model/assets


In [140]:
loaded_model = tf.saved_model.load("./tf_bert_model")

In [142]:
tf_outputs = loaded_model(tf_inputs)

In [146]:
tf_outputs['last_hidden_state']

<tf.Tensor: shape=(1, 8, 768), dtype=float32, numpy=
array([[[ 0.16698128,  0.12934516, -0.28881842, ..., -0.37237155,
          0.30332196,  0.45915374],
        [ 0.54928696,  0.36820287, -0.48872763, ...,  0.05694058,
          0.9668212 ,  0.04267411],
        [ 0.9764617 ,  0.28813305, -0.02539649, ..., -0.780278  ,
          0.7897879 ,  0.158675  ],
        ...,
        [ 0.82881975,  0.32954416, -0.24702382, ..., -0.10966089,
          0.17899686,  0.12884353],
        [ 0.14773697, -0.17064634, -1.1840162 , ..., -0.28526413,
          0.25177714, -0.1390629 ],
        [ 0.9357985 ,  0.07366893, -0.51816905, ...,  0.3814011 ,
         -0.45790046, -0.15736234]]], dtype=float32)>

In [147]:
cls_embedding = tf_outputs['last_hidden_state'][:, 0, :]

In [148]:
cls_embedding[0, 0:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([ 0.16698128,  0.12934516, -0.28881842,  0.01831267, -0.16634007,
       -0.4357676 ,  0.27796954,  0.41088063, -0.31453493,  0.12307151],
      dtype=float32)>

---

In [215]:
import tensorflow as tf
import keras
import tensorflow_text as text
import tensorflow_hub as hub
import keras_hub
import keras_nlp

In [216]:
tf.__version__, keras.__version__

('2.18.0', '3.6.0')

In [217]:
tokenizer = keras_nlp.models.BertTokenizer.from_preset("bert_base_en_uncased")

In [218]:
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_base_en_uncased")

In [219]:
backbone = keras_nlp.models.BertBackbone.from_preset('bert_base_en_uncased', load_weights = True)

In [276]:
#dir(preprocessor)

In [264]:
preprocessor.input_spec

In [275]:
preprocessor.compute_output_shape

<bound method Layer.compute_output_shape of <BertTextClassifierPreprocessor name=bert_text_classifier_preprocessor_4, built=True>>

In [220]:
tokens = tokenizer(prompts[0:1])

In [221]:
encoder_inputs = preprocessor(prompts[0:1])

In [222]:
outputs = backbone(encoder_inputs)

In [223]:
outputs.keys()

dict_keys(['sequence_output', 'pooled_output'])

In [224]:
outputs['pooled_output'].shape

TensorShape([1, 768])

In [225]:
backbone.summary()

Model: "bert_backbone"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ token_ids           │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ token_embedding     │ (None, None, 768) │ 23,440,896 │ token_ids[0][0]   │
│ (ReversibleEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ segment_ids         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ position_embedding  │ (None, None, 768) │    393,216 │ token_embedding[… │
│ (PositionEmbedding) │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ segment_embedding   │ (None, None, 768) │      1,536 │ segment_ids[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embeddings_add      │ (None, None, 768) │          0 │ token_embedding[… │
│ (Add)               │                   │            │ position_embeddi… │
│                     │                   │            │ segment_embeddin… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embeddings_layer_n… │ (None, None, 768) │      1,536 │ embeddings_add[0… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embeddings_dropout  │ (None, None, 768) │          0 │ embeddings_layer… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ padding_mask        │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_layer_0 │ (None, None, 768) │  7,087,872 │ embeddings_dropo… │
│ (TransformerEncode… │                   │            │ padding_mask[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_layer_1 │ (None, None, 768) │  7,087,872 │ transformer_laye… │
│ (TransformerEncode… │                   │            │ padding_mask[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_layer_2 │ (None, None, 768) │  7,087,872 │ transformer_laye… │
│ (TransformerEncode… │                   │            │ padding_mask[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_layer_3 │ (None, None, 768) │  7,087,872 │ transformer_laye… │
│ (TransformerEncode… │                   │            │ padding_mask[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_layer_4 │ (None, None, 768) │  7,087,872 │ transformer_laye… │
│ (TransformerEncode… │                   │            │ padding_mask[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_layer_5 │ (None, None, 768) │  7,087,872 │ transformer_laye… │
│ (TransformerEncode… │                   │            │ padding_mask[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_layer_6 │ (None, None, 768) │  7,087,872 │ transformer_laye… │
│ (TransformerEncode… │                   │            │ padding_mask[0][… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 109,482,240 (417.64 MB)

 Trainable params: 109,482,240 (417.64 MB)

 Non-trainable params: 0 (0.00 B)

In [227]:
#backbone(encoder_inputs)

In [228]:
backbone.save("bert_backbone.keras")

In [229]:
loaded_backbone = keras.saving.load_model("bert_backbone.keras")

In [231]:
#loaded_backbone(encoder_inputs)

In [261]:
#dir(backbone)

In [258]:
backbone.inputs

[<KerasTensor shape=(None, None), dtype=int32, sparse=False, name=padding_mask>,
 <KerasTensor shape=(None, None), dtype=int32, sparse=False, name=segment_ids>,
 <KerasTensor shape=(None, None), dtype=int32, sparse=False, name=token_ids>]

In [253]:
backbone.input_spec

[InputSpec(shape=(None, None), ndim=2),
 InputSpec(shape=(None, None), ndim=2),
 InputSpec(shape=(None, None), ndim=2)]

In [232]:
backbone.export('./bert_backbone', format = 'tf_saved_model', verbose = True)

/home/jupyter/.local/lib/python3.10/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: {'token_ids': 'token_ids', 'segment_ids': 'segment_ids', 'padding_mask': 'padding_mask'}. Received: the structure of inputs=['*', '*', '*']
  warnings.warn(


INFO:tensorflow:Assets written to: ./bert_backbone/assets


INFO:tensorflow:Assets written to: ./bert_backbone/assets


Saved artifact at './bert_backbone'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): List[TensorSpec(shape=(None, None), dtype=tf.int32, name='padding_mask'), TensorSpec(shape=(None, None), dtype=tf.int32, name='segment_ids'), TensorSpec(shape=(None, None), dtype=tf.int32, name='token_ids')]
Output Type:
  Dict[['sequence_output', TensorSpec(shape=(None, None, 768), dtype=tf.float32, name=None)], ['pooled_output', TensorSpec(shape=(None, 768), dtype=tf.float32, name=None)]]
Captures:
  140596254374576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140596253732080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140596253723808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140596253725392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140596253734544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140596253733664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140596390449872: TensorSpec(shape=(), dtype=tf.res

In [241]:
reload_backbone = tf.saved_model.load('./bert_backbone')

In [244]:
reload_backbone.signatures

_SignatureMap({'serve': <ConcreteFunction (*, padding_mask: TensorSpec(shape=(None, None), dtype=tf.int32, name='padding_mask'), segment_ids: TensorSpec(shape=(None, None), dtype=tf.int32, name='segment_ids'), token_ids: TensorSpec(shape=(None, None), dtype=tf.int32, name='token_ids')) -> Dict[['pooled_output', TensorSpec(shape=(None, 768), dtype=tf.float32, name='pooled_output')], ['sequence_output', TensorSpec(shape=(None, None, 768), dtype=tf.float32, name='sequence_output')]] at 0x7FDF26FD86A0>, 'serving_default': <ConcreteFunction (*, padding_mask: TensorSpec(shape=(None, None), dtype=tf.int32, name='padding_mask'), segment_ids: TensorSpec(shape=(None, None), dtype=tf.int32, name='segment_ids'), token_ids: TensorSpec(shape=(None, None), dtype=tf.int32, name='token_ids')) -> Dict[['pooled_output', TensorSpec(shape=(None, 768), dtype=tf.float32, name='pooled_output')], ['sequence_output', TensorSpec(shape=(None, None, 768), dtype=tf.float32, name='sequence_output')]] at 0x7FDF26FDB4

In [245]:
#reload_backbone.serve(encoder_inputs)

In [247]:
encoder_inputs.keys()

dict_keys(['token_ids', 'padding_mask', 'segment_ids'])

In [283]:
# Define input layers for the modified backbone
input_tokens = tf.keras.Input(shape=(None,), dtype=tf.int32, name="token_ids")
input_padding_mask = tf.keras.Input(shape=(None,), dtype=tf.bool, name="padding_mask")
input_segment_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name="segment_ids")
inputs = dict(token_ids = input_tokens, padding_mask = input_padding_mask, segment_ids = input_segment_ids)

In [284]:
# Pass the inputs to the original backbone
outputs = backbone(inputs)

In [285]:
# Create the modified model
modified_backbone = tf.keras.Model(inputs=inputs, outputs=outputs)

In [287]:
backbone.inputs

[<KerasTensor shape=(None, None), dtype=int32, sparse=False, name=padding_mask>,
 <KerasTensor shape=(None, None), dtype=int32, sparse=False, name=segment_ids>,
 <KerasTensor shape=(None, None), dtype=int32, sparse=False, name=token_ids>]

In [288]:
modified_backbone.inputs

[<KerasTensor shape=(None, None), dtype=bool, sparse=False, name=padding_mask>,
 <KerasTensor shape=(None, None), dtype=int32, sparse=False, name=segment_ids>,
 <KerasTensor shape=(None, None), dtype=int32, sparse=False, name=token_ids>]

In [289]:
#modified_backbone(encoder_inputs)

In [290]:
modified_backbone.export('./bert_backbone', format = 'tf_saved_model', verbose = True)

/home/jupyter/.local/lib/python3.10/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: {'token_ids': 'token_ids', 'padding_mask': 'padding_mask', 'segment_ids': 'segment_ids'}. Received: the structure of inputs=['*', '*', '*']
  warnings.warn(


INFO:tensorflow:Assets written to: ./bert_backbone/assets


INFO:tensorflow:Assets written to: ./bert_backbone/assets


Saved artifact at './bert_backbone'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): List[TensorSpec(shape=(None, None), dtype=tf.bool, name='padding_mask'), TensorSpec(shape=(None, None), dtype=tf.int32, name='segment_ids'), TensorSpec(shape=(None, None), dtype=tf.int32, name='token_ids')]
Output Type:
  Dict[['sequence_output', TensorSpec(shape=(None, None, 768), dtype=tf.float32, name=None)], ['pooled_output', TensorSpec(shape=(None, 768), dtype=tf.float32, name=None)]]
Captures:
  140596254374576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140596253732080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140596253723808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140596253725392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140596253734544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140596253733664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140596390449872: TensorSpec(shape=(), dtype=tf.reso

In [304]:
reload_backbone = tf.saved_model.load('./bert_backbone')

In [305]:
reload_backbone.signatures

_SignatureMap({'serving_default': <ConcreteFunction (*, inputs: TensorSpec(shape=(None, None), dtype=tf.bool, name='inputs'), inputs_1: TensorSpec(shape=(None, None), dtype=tf.int32, name='inputs_1'), inputs_2: TensorSpec(shape=(None, None), dtype=tf.int32, name='inputs_2')) -> Dict[['pooled_output', TensorSpec(shape=(None, 768), dtype=tf.float32, name='pooled_output')], ['sequence_output', TensorSpec(shape=(None, None, 768), dtype=tf.float32, name='sequence_output')]] at 0x7FDF4C4774C0>})

In [306]:
reload_backbone(encoder_inputs)

TypeError: '_UserObject' object is not callable

In [309]:
model = reload_backbone.signatures['serving_default']

In [310]:
model(encoder_inputs)

TypeError: Binding inputs to tf.function failed due to `too many positional arguments`. Received args: ({'token_ids': <tf.Tensor: shape=(1, 512), dtype=int32, numpy=
array([[ 101, 2129, 2079, 2017, 2377, 3598, 1029,  102,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 

In [314]:
tf.saved_model.save(
    modified_backbone,
    './bert_backbone',
    signatures=modified_backbone.call.get_concrete_function(
        {
            "token_ids": tf.TensorSpec(shape=(None, None), dtype=tf.int32),
            "padding_mask": tf.TensorSpec(shape=(None, None), dtype=tf.bool),
            "segment_ids": tf.TensorSpec(shape=(None, None), dtype=tf.int32),
        }
    ),
)

AttributeError: 'function' object has no attribute 'get_concrete_function'

In [316]:
tf.saved_model.save(
    modified_backbone,
    "./albert_backbone",
    signatures={
        'serving_default': modified_backbone.call(  # Call the method directly
            {
                "token_ids": tf.TensorSpec(shape=(None, None), dtype=tf.int32),
                "padding_mask": tf.TensorSpec(shape=(None, None), dtype=tf.bool),
                "segment_ids": tf.TensorSpec(shape=(None, None), dtype=tf.int32),
            }
        )
    },
)

ValueError: Attempt to convert a value (TensorSpec(shape=(None, None), dtype=tf.bool, name=None)) with an unsupported type (<class 'tensorflow.python.framework.tensor.TensorSpec'>) to a Tensor.

In [319]:
falcon_lm = keras_hub.models.FalconCausalLM.from_preset("falcon_refinedweb_1b_en")
falcon_lm.generate("I want to say", max_length=30)

100%|██████████| 557/557 [00:00<00:00, 793kB/s]


100%|██████████| 4.89G/4.89G [01:35<00:00, 55.1MB/s]


100%|██████████| 454/454 [00:00<00:00, 310kB/s]


100%|██████████| 0.99M/0.99M [00:00<00:00, 4.77MB/s]


100%|██████████| 446k/446k [00:00<00:00, 3.06MB/s]
I0000 00:00:1732309199.252247  449453 service.cc:148] XLA service 0x5629d97709b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732309199.256200  449453 service.cc:156]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1732309199.500683  449453 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


'I want to say a few words about the new “Sleeping Giant” series of novels by John G. Hemry (the'

In [320]:
dir(falcon_lm)

['__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_trackable_child',
 '_add_variable_with_custom_getter',
 '_adjust_input_rank',
 '_allow_non_tensor_positional_args',
 '_api_export_path',
 '_api_export_symbol_id',
 '_assert_compile_called',
 '_assert_input_compatibility',
 '_assign_variable_values',
 '_auto_config',
 '_backbone',
 '_build_by_run_for_kwargs',
 '_build_by_run_for_single_pos_arg',
 '_build_cache',
 '_build_shapes_dict',
 '_call_has_mask_arg',
 '_call_has_training_arg',
 '_call_signature',
 '_called',
 '_check_quantize_args',
 '_check_super_called',
 '_checkpoint_adapter',
 '_checkpoint_dependencies',
 '_clear_losses',
 

In [321]:
falcon_lm.summary()

Preprocessor: "falcon_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ falcon_tokenizer (FalconTokenizer)                            │                       Vocab size: 50,257 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "falcon_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ falcon_backbone               │ (None, None, 2048)        │   1,311,625,216 │ padding_mask[0][0],        │
│ (FalconBackbone)              │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 50304)       │     103,022,592 │ falcon_backbone[0][0]      │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 1,311,625,216 (4.89 GB)

 Trainable params: 1,311,625,216 (4.89 GB)

 Non-trainable params: 0 (0.00 B)

In [323]:
falcon_lm.preprocessor('This is a sentence')

({'token_ids': <tf.Tensor: shape=(1024,), dtype=int32, numpy=array([50256,  1212,   318, ...,     0,     0,     0], dtype=int32)>,
  'padding_mask': <tf.Tensor: shape=(1024,), dtype=bool, numpy=array([ True,  True,  True, ..., False, False, False])>},
 <tf.Tensor: shape=(1024,), dtype=int32, numpy=array([1212,  318,  257, ...,    0,    0,    0], dtype=int32)>,
 <tf.Tensor: shape=(1024,), dtype=bool, numpy=array([ True,  True,  True, ..., False, False, False])>)

In [324]:
falcon_lm(falcon_lm.preprocessor('This is a sentence'))

ValueError: Layer "falcon_causal_lm" expects 2 input(s), but it received 4 input tensors. Inputs received: [<tf.Tensor: shape=(1024,), dtype=bool, numpy=array([ True,  True,  True, ..., False, False, False])>, <tf.Tensor: shape=(1024,), dtype=int32, numpy=array([50256,  1212,   318, ...,     0,     0,     0], dtype=int32)>, <tf.Tensor: shape=(1024,), dtype=int32, numpy=array([1212,  318,  257, ...,    0,    0,    0], dtype=int32)>, <tf.Tensor: shape=(1024,), dtype=bool, numpy=array([ True,  True,  True, ..., False, False, False])>]